In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import pickle

## Step 1: Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('Resources/kc_house_data_test_features.csv', index_col=0)

final_scaler = pickle.load(open('scaler.pickle','rb'))
final_model = pickle.load(open('model.pickle','rb'))

In [3]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [4]:
##Only run this cell once!

#Create new column 'age' for age of house
df['age'] = 2020 - df.yr_built

#Create new column 'last_renovted' for how long ago since last renovation
df['last_ren'] = np.where(df['yr_renovated'] > 0, (2020 - df['yr_renovated']), 0)

#Create new column 'last_ren2' for how long (years) since last renovation
df['last_ren2'] = np.where(df['yr_renovated'] > 0, (2020 - df['yr_renovated']), (2020 - df['yr_built']))

#Create 0/1 new column 'renovated'
df['renovated'] = np.where(df['yr_renovated']>0 , 1, 0)

#Create dummy columns using 'renovated'
df = pd.get_dummies(df, columns=['renovated'])

#Create dummy variable 'basement' y/n - 6806 with basement, 10484 without
df['basement'] = np.where(df['sqft_basement']>0 , 1, 0)

#Create a new column 'neighbors_compared' with sqft_living - sqft_living15
df['neighbors_compared'] = df.sqft_living - df.sqft_living15


#Fixing outliers
bathroom_median = df.bathrooms.median()
df.loc[df.bathrooms == 0, 'bathrooms'] = np.nan
df.fillna(bathroom_median,inplace=True)

#drop 33 bedrooms outlier
df = df[df.bedrooms != 33]

In [5]:
df = df.drop(['date','lat', 'long', 'zipcode'], axis=1)

In [6]:
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,age,last_ren,last_ren2,renovated_0,renovated_1,basement,neighbors_compared
0,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,2020,10918,53,0,53,1,0,1,250
1,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,2020,10918,53,0,53,1,0,1,250
2,3630020380,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,1470,1576,15,0,15,1,0,1,0
3,1771000290,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,1160,10565,44,0,44,1,0,1,120
4,5126310470,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,2830,7916,15,0,15,1,0,0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4323 entries, 0 to 4322
Data columns (total 23 columns):
id                    4323 non-null int64
bedrooms              4323 non-null int64
bathrooms             4323 non-null float64
sqft_living           4323 non-null int64
sqft_lot              4323 non-null int64
floors                4323 non-null float64
waterfront            4323 non-null int64
view                  4323 non-null int64
condition             4323 non-null int64
grade                 4323 non-null int64
sqft_above            4323 non-null int64
sqft_basement         4323 non-null int64
yr_built              4323 non-null int64
yr_renovated          4323 non-null int64
sqft_living15         4323 non-null int64
sqft_lot15            4323 non-null int64
age                   4323 non-null int64
last_ren              4323 non-null int64
last_ren2             4323 non-null int64
renovated_0           4323 non-null uint8
renovated_1           4323 non-null uint8
basemen

In [8]:
# transformed_holdout = final_scaler(holdout)
#Fixing outliers
bathroom_median = df.bathrooms.median()
df.loc[df.bathrooms == 0, 'bathrooms'] = np.nan
df.fillna(bathroom_median,inplace=True)

#drop 33 bedrooms outlier
df = df[df.bedrooms != 33]

In [20]:
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,age,last_ren,last_ren2,renovated_0,renovated_1,basement,neighbors_compared
0,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,2020,10918,53,0,53,1,0,1,250
1,1974300020,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,2020,10918,53,0,53,1,0,1,250
2,3630020380,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,1470,1576,15,0,15,1,0,1,0
3,1771000290,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,1160,10565,44,0,44,1,0,1,120
4,5126310470,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,2830,7916,15,0,15,1,0,0,0


In [21]:
df.shape

(4323, 23)

In [18]:
transformed_holdout =pd.DataFrame(data=final_scaler.transform(df),columns=df.columns)


ValueError: operands could not be broadcast together with shapes (4323,23) (10,) (4323,23) 

## Step 3: Predict the holdout set

In [ ]:
final_answers = final_model.predict(transformed_holdout)

In [ ]:
transformed_holdout = pd.DataFrame(data=final_scaler.transform(df),columns=df.columns)

In [ ]:

final_answers = final_model.predict(transformed_holdout)

In [ ]:
winner.coef_

## Step 4: Export your predictions

In [ ]:
# final_answer.to_csv('housing_preds_cierra_andaur.csv')